In [ ]:
!unzip train.zip -d train

Archive:  train.zip
   creating: train/train/
   creating: train/train/with_mask/
  inflating: train/train/with_mask/-110603108-gettyimages-533567012.jpg  
  inflating: train/train/with_mask/-110695084-mask2.jpg  
  inflating: train/train/with_mask/0-with-mask.jpg  
  inflating: train/train/with_mask/0000305-62360-halyard-health-pfr-p3-respirator-face-mask-ffp.jpeg  
  inflating: train/train/with_mask/0000444-48296-halyard-health-surgical-mask-teddy-bear-print.jpeg  
  inflating: train/train/with_mask/007563.jpg  
  inflating: train/train/with_mask/0779db20580fc7b054ad6b1b7bd9fb35.jpeg  
  inflating: train/train/with_mask/1-respirator-mask.jpg  
  inflating: train/train/with_mask/10-with-mask.jpg  
  inflating: train/train/with_mask/100-with-mask.jpg  
  inflating: train/train/with_mask/101-with-mask.jpg  
  inflating: train/train/with_mask/103-with-mask.jpg  
  inflating: train/train/with_mask/104-with-mask.jpg  
  inflating: train/train/with_mask/105-with-mask.jpg  
  inflating: trai

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np

In [ ]:
model = Sequential()
model.add(Conv2D(64,(3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(2))
model.add(Activation('softmax'))
# compile the model
model.compile(optimizer='adam', loss = 'binary_crossentropy', 
metrics=['accuracy'])
model.summary()

"""model = Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])"""

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 64)      1792      
                                                                 
 activation (Activation)     (None, 148, 148, 64)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 256)       147712    
                                                                 
 activation_1 (Activation)   (None, 72, 72, 256)       0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 256)      0         
 2D)                                                    

"model = Sequential([\n    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),\n    MaxPooling2D(2,2),\n    \n    Conv2D(100, (3,3), activation='relu'),\n    MaxPooling2D(2,2),\n    \n    Flatten(),\n    Dropout(0.5),\n    Dense(50, activation='relu'),\n    Dense(2, activation='softmax')\n])\nmodel.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])"

In [ ]:
TRAINING_DIR = "train/train"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150))
VALIDATION_DIR = "test/test"
validation_datagen = ImageDataGenerator(rescale=1.0/255)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

Found 1315 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


In [ ]:
history = model.fit_generator(train_generator,
                              epochs=10,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  after removing the cwd from sys.path.


Epoch 1/10
 22/132 [====>.........................] - ETA: 3:03 - loss: 0.7036 - accuracy: 0.4818

KeyboardInterrupt: ignored

In [ ]:
from google.colab import files
files.download('model2-009.model') 

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")

In [ ]:
model.save_weights('weights.h5')

In [ ]:
model.save('architecure.h5')